In [ ]:
import numpy as np
import pandas as pd
import nltk
import time
import openai
openai.api_key = "TYPE HERE YOUR API KEY"

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_colwidth', None)

## Main tasks

In [ ]:
#Create a new dataframe
LI_PM_jobad = pd.read_csv('LI_PM_jobDesc.csv', delimiter = ',')
df = pd.DataFrame()
df['companyName'] = LI_PM_jobad['companyName']
df['jobDescription'] = LI_PM_jobad['jobDescription']
df['mainTasks'] = ''

#iterate through the dataframe
for index, row in df.iterrows():
    
    prompt_mainTasks = row['jobDescription'] + "/n --- /n This is a product manager job description.\
        Make a brief summary of what the employee's main tasks will be in this job role.\
        List only the specific duties related to the job itself and\
        do not include general environment characteristics or soft skills or employee benefits.\
        Please, use only keywords (or short expressions) in your answer, do not construct full sentences.\
        Answer the question as truthfully as possible using the provided job description.\
        If there are no clear information abput main tasks in the description,\
        use only this value: 'Unknown' and don't use whole sentences."
    
    response_mainTasks = openai.ChatCompletion.create(
        model = "gpt-4",
        messages = [
            {"role": "system", "content": "You are a helpful data science assistant."},
            {"role": "user", "content": prompt_mainTasks}],
        max_tokens=100,
        temperature=0)
    
    message_mainTasks = response_mainTasks['choices'][0]['message']['content'].strip()

    df.loc[index, 'mainTasks'] = message_mainTasks
    
    time.sleep(0.5)
        
    if index % 10 == 0:
        print(f"Processed row {index} for company {row['companyName']} with main tasks: {message_mainTasks}")
        
#print results
df[['companyName', 'mainTasks']]

## Further analysis with chatGPT

At first, I classified the text fragments collected by the AI from the 212 job advertisements into 7 large groups. Then, I copied these groups one by one to ChatGPT and asked it to help me with the analysis using the following prompt:

"I will show some job advertisement details about main tasks given by employers - from now on, I will refer to this group of advertisements as jobAd_n.
Tasks:
1) Combine similar tasks in the jobAd_n advertisement group!
2) Then, collect the 15 most frequently occurring tasks from this jobAd_n group!
3) Finally, make a table of how many advertisements appeared in the jobAd_n group!"

Then, I saved the table created by ChatGPT for myself and performed the same analysis with all advertisement groups. Finally, I copied the tables created after all 7 groups and asked ChatGPT to combine similar tasks and make a last combined table.